Preparation for Classification

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

import folium # plotting library

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
df = pd.read_excel('full_list.xlsx', index_col=0)

In [3]:
whole_list = df

Explore the spots

In [4]:
# Define Foursquare Credentials and Version

CLIENT_ID = '5P4U2KHYRYF4YZIKIGSGT45I2ILKGBKE5TKNH2LIDZAPLO3J' 
CLIENT_SECRET = 'DWF3I3K2GHAB4WRU1Q22FWD5ZCKO4VXFFZNY1CKUPJHEYH5K' 
VERSION = '20180605' # Foursquare API version


In [5]:
# Function to get the top 100 venues that are in a neighborhood within a radius of 500 meters

LIMIT = 100 # top 100 venues

def getNearbyVenues(m_ids, s_ids, names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for m_id, s_id, name, lat, lng in zip(m_ids, s_ids, names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
                m_id,
                s_id,
                name, 
                lat, 
                lng, 
                v['venue']['id'],
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']) for v in results])

    
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = [
                  'm_id',
                  's_id',
                  'Type', 
                  'Latitude', 
                  'Longitude', 
                  'Venue ID',
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)


In [6]:
# Run the above function on each spots and create a new dataframe called venues
import time
start = time.time()
venues = getNearbyVenues(
                                m_ids=df['M_id'],
                                s_ids=df['S_id'],
                                names=df['Type'],
                                latitudes=df['Latitude'],
                                longitudes=df['Longitude']
                                )
end = time.time()
end - start

103.25455856323242

In [7]:
venues.to_excel('venues.xlsx')